In [5]:
# 앙상블 검색기 (Ensemble Retriever)
# EnsembleRetriever는 여러 retriever를 입력으로 받아 get_relevant_documents() 메서드의 결과를 앙상블하고, 
# Reciprocal Rank Fusion 알고리즘을 기반으로 결과를 재순위화합니다.
# 서로 다른 알고리즘의 장점을 활용함으로써, EnsembleRetriever는 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.
# 가장 일반적인 패턴은 sparse retriever (예: BM25)와 dense retriever (예: embedding similarity)를 결합하는 것인데, 
# 이는 두 retriever의 장점이 상호 보완적이기 때문입니다. 이를 "hybrid search" 라고도 합니다.
# Sparse retriever는 키워드를 기반으로 관련 문서를 찾는 데 효과적이며, dense retriever는 의미적 유사성을 기반으로 관련 문서를 찾는 데 효과적입니다.

%pip install -qU rank_bm25 deeplake > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[OPENAI_API_KEY]\n{os.environ['OPENAI_API_KEY']}\n")
print(f"[HUGGINGFACEHUB_API_TOKEN]\n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

In [3]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 비타민 별 섭취할 수 있는 음식 정보
doc_list_1 = [
    "비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.",
    "비타민B : 전곡물, 콩, 견과류, 육류 등 다양한 식품에서 찾을 수 있습니다.",
    "비타민C : 오렌지, 키위, 딸기, 브로콜리, 피망 등의 과일과 채소에 많이 들어 있습니다.",
    "비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.",
    "비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.",
]

# 비타민 별 효능 정보
doc_list_2 = [
    "비타민A : 시력과 피부 건강을 지원합니다.",
    "비타민B : 에너지 대사와 신경계 기능을 돕습니다.",
    "비타민C : 면역 체계를 강화하고 콜라겐 생성을 촉진합니다.",
    "비타민D : 뼈 건강과 면역 체계를 지원합니다.",
    "비타민E : 항산화 작용을 통해 세포를 보호합니다.",
]

# bm25 retriever초기화합니다.
bm25_retriever = BM25Retriever.from_texts(
    # doc_list_1의 텍스트와 메타데이터를 사용하여 BM25Retriever를 초기화합니다.
    doc_list_1,
    metadatas=[{"source": 1}] * len(doc_list_1),
)

bm25_retriever.k = 2  # BM25Retriever의 검색 결과 개수를 2로 설정합니다.

# faiss retriever를 초기화
embedding = OpenAIEmbeddings()  # OpenAI 임베딩을 사용합니다.
faiss_vectorstore = FAISS.from_texts(
    # doc_list_2의 텍스트와 임베딩, 메타데이터를 사용하여 FAISS 벡터 저장소를 초기화합니다.
    doc_list_2,
    embedding,
    metadatas=[{"source": 2}] * len(doc_list_2),
)
# FAISS 벡터 저장소를 사용하여 retriever를 생성하고, 검색 결과 개수를 2로 설정합니다.
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    # BM25Retriever와 FAISS retriever를 사용하여 EnsembleRetriever를 초기화하고, 각 retriever의 가중치를 0.6:0.4로 설정합니다.
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.6, 0.4],
    search_type="mmr",
)


In [4]:
# 검색 결과 문서를 가져옵니다.
query = "비타민A 의 효능은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Ensemble Retriever]
 [Document(page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.', metadata={'source': 1}), Document(page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.', metadata={'source': 1}), Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2}), Document(page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.', metadata={'source': 2})]

[BM25 Retriever]
 [Document(page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.', metadata={'source': 1}), Document(page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.', metadata={'source': 1})]

[FAISS Retriever]
 [Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2}), Document(page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.', metadata={'source': 2})]

